In [1]:
# 1. Data Cleaning

In [2]:
import math
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file

# For visvalization
import matplotlib.pyplot as plt
import seaborn as sns
# import geopandas as gpd

%matplotlib inline

In both data , we only need:
1. Country name	
2. year	
3. Life Ladder	
4. Log GDP per capita	
5. Social support	
6. Healthy life expectancy at birth	
7. Freedom to make life choices	
8. Generosity	
9. Perceptions of corruption

Clear the Time Series Data first
1. Removed year 2005
2. Dropped 'Positive affect','Negative affect'

In [3]:
# Reading the csv files
df = pd.read_csv('../dataset/data/world-happiness-report-2021.csv')
df_ts = pd.read_csv('../dataset/data/world-happiness-report.csv')

In [4]:
#2006 - 2021
init = 2006
years = []
for i in range(16):
    years.append(init)
    init += 1
    
print(years)

[2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]


In [5]:
for year in years:
    size = len(df_ts[df_ts['year'] == year])
    print(year," : ",size)

2006  :  89
2007  :  102
2008  :  110
2009  :  114
2010  :  124
2011  :  146
2012  :  142
2013  :  137
2014  :  145
2015  :  143
2016  :  142
2017  :  147
2018  :  142
2019  :  144
2020  :  95
2021  :  0


In [6]:
#Remove 2005 data
df_ts = df_ts[df_ts['year'] != 2005]
drop_list = ['Positive affect','Negative affect']
df_ts = df_ts.drop(drop_list, axis = 1)
df_ts

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776
...,...,...,...,...,...,...,...,...,...
1944,Zimbabwe,2016,3.735,7.984,0.768,54.40,0.733,-0.095,0.724
1945,Zimbabwe,2017,3.638,8.016,0.754,55.00,0.753,-0.098,0.751
1946,Zimbabwe,2018,3.616,8.049,0.775,55.60,0.763,-0.068,0.844
1947,Zimbabwe,2019,2.694,7.950,0.759,56.20,0.632,-0.064,0.831


Clear the 2021 only data
1 . Rename the column as same as the Time Series Data
2 . Added 'year' data to 2021 data
3 . Remove Unecessary Columns

In [7]:
# drop_list = ['Standard error of ladder score','Regional indicator','Standard error of ladder score',
#             ]
columns = list(df_ts.columns)
print(columns)

df = df.rename(columns={'Ladder score': 'Life Ladder'})
df = df.rename(columns={'Logged GDP per capita': 'Log GDP per capita'})
df = df.rename(columns={'Healthy life expectancy': 'Healthy life expectancy at birth'})

df_2021 = df
row_num = df_2021.shape[0]
year_2021 = []

for i in range(row_num):
    year_2021.append(2021)

df_2021['year'] = year_2021

df_2021 = df_2021[columns]

df_2021 = df[columns]

df_2021

['Country name', 'year', 'Life Ladder', 'Log GDP per capita', 'Social support', 'Healthy life expectancy at birth', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption']


,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption
0,Finland,2021,7.842,10.775,0.954,72.000,0.949,-0.098,0.186
1,Denmark,2021,7.620,10.933,0.954,72.700,0.946,0.030,0.179
2,Switzerland,2021,7.571,11.117,0.942,74.400,0.919,0.025,0.292
3,Iceland,2021,7.554,10.878,0.983,73.000,0.955,0.160,0.673
4,Netherlands,2021,7.464,10.932,0.942,72.400,0.913,0.175,0.338
...,...,...,...,...,...,...,...,...,...
144,Lesotho,2021,3.512,7.926,0.787,48.700,0.715,-0.131,0.915
145,Botswana,2021,3.467,9.782,0.784,59.269,0.824,-0.246,0.801
146,Rwanda,2021,3.415,7.676,0.552,61.400,0.897,0.061,0.167
147,Zimbabwe,2021,3.145,7.943,0.750,56.201,0.677,-0.047,0.821


Combine the 2021 data into time series
Save the data and rearrange the index 

In [8]:
frames = [df_2021, df_ts]
result = pd.concat(frames)
result = result.sort_values(['Country name', 'year'], ascending=[True, True])

In [9]:
pd.set_option('display.max_rows', None)
result.to_csv('../dataset/clean_data/2006-2021-world-happiness-report.csv', index=False)
result = pd.read_csv('../dataset/clean_data/2006-2021-world-happiness-report.csv')
df_ts = result
df_ts

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption
0,Afghanistan,2008,3.724,7.370,0.451,50.800,0.718,0.168,0.882
1,Afghanistan,2009,4.402,7.540,0.552,51.200,0.679,0.190,0.850
2,Afghanistan,2010,4.758,7.647,0.539,51.600,0.600,0.121,0.707
3,Afghanistan,2011,3.832,7.620,0.521,51.920,0.496,0.162,0.731
4,Afghanistan,2012,3.783,7.705,0.521,52.240,0.531,0.236,0.776
5,Afghanistan,2013,3.572,7.725,0.484,52.560,0.578,0.061,0.823
6,Afghanistan,2014,3.131,7.718,0.526,52.880,0.509,0.104,0.871
7,Afghanistan,2015,3.983,7.702,0.529,53.200,0.389,0.080,0.881
8,Afghanistan,2016,4.220,7.697,0.559,53.000,0.523,0.042,0.793
9,Afghanistan,2017,2.662,7.697,0.491,52.800,0.427,-0.121,0.954


In [10]:
df_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2071 entries, 0 to 2070
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Country name                      2071 non-null   object 
 1   year                              2071 non-null   int64  
 2   Life Ladder                       2071 non-null   float64
 3   Log GDP per capita                2035 non-null   float64
 4   Social support                    2058 non-null   float64
 5   Healthy life expectancy at birth  2016 non-null   float64
 6   Freedom to make life choices      2041 non-null   float64
 7   Generosity                        2008 non-null   float64
 8   Perceptions of corruption         1963 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 145.7+ KB


In [11]:
df_ts.describe()

,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption
count,2071.000000,2071.000000,2035.000000,2058.000000,2016.000000,2041.000000,2008.000000,1963.000000
mean,2013.883148,5.458693,9.363167,0.811598,63.431241,0.745073,-0.001155,0.746030
std,4.401675,1.109517,1.156030,0.118196,7.489589,0.140857,0.161339,0.186119
min,2006.000000,2.375000,6.635000,0.290000,32.300000,0.258000,-0.335000,0.035000
25%,2010.000000,4.640000,8.460000,0.748000,58.695000,0.651000,-0.115000,0.690000
50%,2014.000000,5.385000,9.458000,0.833000,65.222500,0.766000,-0.027000,0.801000
75%,2018.000000,6.261000,10.347000,0.904750,68.600000,0.858000,0.089000,0.869000
max,2021.000000,7.971000,11.648000,0.987000,77.100000,0.985000,0.698000,0.983000


In [12]:
df_ts.isnull().sum()

Country name                          0
year                                  0
Life Ladder                           0
Log GDP per capita                   36
Social support                       13
Healthy life expectancy at birth     55
Freedom to make life choices         30
Generosity                           63
Perceptions of corruption           108
dtype: int64

In [13]:
df_ts['Country name'].unique().shape

(166,)

Check whether 
1. each country at least contains 50% of the data, 
Since we only consider the data from 2006-2021 , there are 16 data in total, the country must at least have 8 data.

2. whether the columns are completely empty or not
Since some columns are actually empty, For instance the GDP for Hong Kong.
If it happens, we will remove the data of this country.

3. if the columns are not empty, check if it at least contains 50% of data .However, we will remove the data later 

In [14]:
# #2006 - 2021
# init = 2006
# years = []
# for i in range(16):
#     years.append(init)
#     init += 1
    
# print(years)

In [15]:
countries = df_ts["Country name"].unique()
columns = list(df_ts.columns)
print(columns)

['Country name', 'year', 'Life Ladder', 'Log GDP per capita', 'Social support', 'Healthy life expectancy at birth', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption']


In [16]:
#Create Empty Dataframe and dictionaries
df_mean = pd.DataFrame()
dict_mean = {}

countries_low_freq = []
countries_null = []
countries_low_freq_cols = []

for col_item in columns:
    dict_mean[col_item] = []

dict_mean['Country name'] = countries

# df_china = df_ts[df_ts['Country name'] == 'China'] 

for country in countries:
    df_bycountry = df_ts[df_ts['Country name'] == country] 
    data_num = df_bycountry.shape[0]
    for col_item in columns:
        if col_item != "Country name":
            mean = df_bycountry[col_item].mean()
            col_data_num = df_bycountry[col_item].shape[0] - df_bycountry[col_item].isnull().sum()
            dict_mean[col_item].append(mean)

            #1. each country at least contains 50% of the data,    
            if data_num < ( 16 *0.5 ):
                countries_low_freq.append(country)
            #2. whether the columns are completely empty or not
            if math.isnan(mean):
                countries_null.append(country)
            #3. 3. if the columns are not empty, check if the chosen columns at least contains 50% of data
            chosen_cols = ["Healthy life expectancy at birth","Social support","Log GDP per capita"]
            if col_data_num < (16 *0.5) and (col_item in chosen_cols):
                countries_low_freq_cols.append(country)
                
for col_item in columns:
     df_mean[col_item] = dict_mean[col_item]

# df_china['Freedom to make life choices'].shape[0] - df_china['Freedom to make life choices'].isnull().sum()
# df_china

In [17]:
#Countries that has low frequency of data(50%)
countries_low_freq = list(set(countries_low_freq))
print(len(countries_low_freq),"Countries that have low frequency of data(50%): "
      ,countries_low_freq)

25 Countries that have low frequency of data(50%):  ['Somalia', 'Maldives', 'Central African Republic', 'Gambia', 'Lesotho', 'Syria', 'Comoros', 'Belize', 'Cuba', 'Oman', 'Suriname', 'Djibouti', 'Somaliland region', 'Burundi', 'Qatar', 'South Sudan', 'Swaziland', 'Bhutan', 'Angola', 'Namibia', 'Libya', 'Trinidad and Tobago', 'Sudan', 'Jamaica', 'Guyana']


In [18]:
#Countries that contain null value
null_countries_list = list(set(countries_null))
print(len(null_countries_list),"Countries that have null values: "
      ,null_countries_list)

5 Countries that have null values:  ['Somalia', 'Cuba', 'Oman', 'South Sudan', 'Somaliland region']


In [19]:
#Countries that have column items with low frequency of data(50%)
countries_low_freq_cols = list(set(countries_low_freq_cols))
print(len(countries_low_freq_cols),"Countries that have column items with low frequency of data(50%): "
      ,countries_low_freq_cols)

30 Countries that have column items with low frequency of data(50%):  ['Somalia', 'Maldives', 'Central African Republic', 'Gambia', 'Lesotho', 'North Cyprus', 'Syria', 'Comoros', 'Belize', 'Cuba', 'Oman', 'Suriname', 'Djibouti', 'Somaliland region', 'Kosovo', 'Palestinian Territories', 'Burundi', 'Qatar', 'South Sudan', 'Swaziland', 'Bhutan', 'Taiwan Province of China', 'Hong Kong S.A.R. of China', 'Angola', 'Namibia', 'Libya', 'Trinidad and Tobago', 'Sudan', 'Jamaica', 'Guyana']


In [20]:
unwanted_countries_list = null_countries_list + countries_low_freq 
unwanted_countries_list = list(set(unwanted_countries_list))

print("We Filtered out ", len(unwanted_countries_list) ," countries")
print(unwanted_countries_list)

We Filtered out  25  countries
['Somalia', 'Maldives', 'Central African Republic', 'Gambia', 'Lesotho', 'Syria', 'Comoros', 'Belize', 'Cuba', 'Oman', 'Suriname', 'Djibouti', 'Somaliland region', 'Burundi', 'Qatar', 'South Sudan', 'Swaziland', 'Bhutan', 'Angola', 'Namibia', 'Libya', 'Trinidad and Tobago', 'Sudan', 'Jamaica', 'Guyana']


In [21]:
#Countries that we want

countries_wanted_list = list(countries)

for unwanted_country in unwanted_countries_list:
    if unwanted_country in countries_wanted_list:
        countries_wanted_list.remove(unwanted_country)
        
print("We now have ", len(countries_wanted_list) ," countries")
print(countries_wanted_list)

We now have  141  countries
['Afghanistan', 'Albania', 'Algeria', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Benin', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Cambodia', 'Cameroon', 'Canada', 'Chad', 'Chile', 'China', 'Colombia', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Ethiopia', 'Finland', 'France', 'Gabon', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guinea', 'Haiti', 'Honduras', 'Hong Kong S.A.R. of China', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Ivory Coast', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kosovo', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon', 'Liberia', 'Lithuania', 'Luxembourg', 'Madagascar', 'Malawi', 'Malaysia', 'Mali', 'Malta', 'Mauritania', 'Mauritius'

In [22]:
df_ts_filtered = df_ts[df_ts['Country name'].isin(countries_wanted_list) ]

In [23]:
df_ts_filtered 

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption
0,Afghanistan,2008,3.724,7.370,0.451,50.800,0.718,0.168,0.882
1,Afghanistan,2009,4.402,7.540,0.552,51.200,0.679,0.190,0.850
2,Afghanistan,2010,4.758,7.647,0.539,51.600,0.600,0.121,0.707
3,Afghanistan,2011,3.832,7.620,0.521,51.920,0.496,0.162,0.731
4,Afghanistan,2012,3.783,7.705,0.521,52.240,0.531,0.236,0.776
5,Afghanistan,2013,3.572,7.725,0.484,52.560,0.578,0.061,0.823
6,Afghanistan,2014,3.131,7.718,0.526,52.880,0.509,0.104,0.871
7,Afghanistan,2015,3.983,7.702,0.529,53.200,0.389,0.080,0.881
8,Afghanistan,2016,4.220,7.697,0.559,53.000,0.523,0.042,0.793
9,Afghanistan,2017,2.662,7.697,0.491,52.800,0.427,-0.121,0.954


In [24]:
#Replace all the null value with mean
#Current null value:
print("Before Removing Null Value")
df_ts_filtered.isnull().sum()

Before Removing Null Value


Country name                         0
year                                 0
Life Ladder                          0
Log GDP per capita                  24
Social support                       9
Healthy life expectancy at birth    51
Freedom to make life choices        28
Generosity                          50
Perceptions of corruption           99
dtype: int64

In [25]:
# df_mean[df_mean['Country name'] == 'Afghanistan']['year'].values[0]

# print("After Removing Null Value")
# for ind in df_ts_filtered.index:
#     for col_item in columns:
#         if col_item != 'Country name':
#             if math.isnan(df_ts_filtered[col_item][ind]):
#                 country =  df_ts_filtered['Country name'][ind]
#                 df_ts_filtered[col_item][ind] = df_mean[df_mean['Country name'] == country][col_item].values[0]

df_ts_filtered.isnull().sum()

Country name                         0
year                                 0
Life Ladder                          0
Log GDP per capita                  24
Social support                       9
Healthy life expectancy at birth    51
Freedom to make life choices        28
Generosity                          50
Perceptions of corruption           99
dtype: int64

In [26]:
df_ts_filtered 

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption
0,Afghanistan,2008,3.724,7.370,0.451,50.800,0.718,0.168,0.882
1,Afghanistan,2009,4.402,7.540,0.552,51.200,0.679,0.190,0.850
2,Afghanistan,2010,4.758,7.647,0.539,51.600,0.600,0.121,0.707
3,Afghanistan,2011,3.832,7.620,0.521,51.920,0.496,0.162,0.731
4,Afghanistan,2012,3.783,7.705,0.521,52.240,0.531,0.236,0.776
5,Afghanistan,2013,3.572,7.725,0.484,52.560,0.578,0.061,0.823
6,Afghanistan,2014,3.131,7.718,0.526,52.880,0.509,0.104,0.871
7,Afghanistan,2015,3.983,7.702,0.529,53.200,0.389,0.080,0.881
8,Afghanistan,2016,4.220,7.697,0.559,53.000,0.523,0.042,0.793
9,Afghanistan,2017,2.662,7.697,0.491,52.800,0.427,-0.121,0.954


In [27]:
df_ts_filtered.to_csv('../dataset/clean_data/clean-world-happiness-report.csv', index=False)